# LLaVA-OneVision Fine-Tuning for Weather Satellite Analysis

## Overview
This notebook demonstrates fine-tuning the **LLaVA-OneVision-7B-OV** vision-language model for weather satellite image analysis using advanced Parameter-Efficient Fine-Tuning (PEFT) techniques.

### Key Features
- **Model**: LLaVA-OneVision-7B-OV
- **Techniques**: DoRA, AdaLoRA, and QLoRA
- **Dataset**: Weather Satellite Images with captions
- **Framework**: HuggingFace Transformers + TRL + PEFT

### What is LLaVA-OneVision?
LLaVA-OneVision is a state-of-the-art vision-language model that:
- Understands and generates natural language about images
- Combines visual perception with language understanding
- Provides detailed descriptions of visual content



---

## 1. Environment Setup & Installation

### Kaggle Dataset Import
Import the weather satellite dataset from Kaggle Hub.

In [ ]:
import kagglehub
azdinsahir_wsi_data_pfe_path = kagglehub.dataset_download('azdinsahir/wsi-data-pfe')

print('Data source import complete.')


### Install Required Libraries
Installing LLaVA-OneVision dependencies including:
- `transformers`: HuggingFace library for model loading
- `bitsandbytes`: For 4-bit quantization
- `peft`: Parameter-Efficient Fine-Tuning library
- `trl`: Transformer Reinforcement Learning
- `wandb`: Experiment tracking

In [ ]:
# ====================================================================
# CELL 1: LLaVA-OneVision Setup
# ====================================================================

# Install required packages
!pip install -q transformers>=4.44.0
!pip install -q torch torchvision torchaudio
!pip install -q bitsandbytes accelerate
!pip install -q peft trl
!pip install -q wandb
!pip install -q pillow numpy
!pip install -q sentencepiece protobuf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 93.5 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 28.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 79.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 22.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━

### Weights & Biases Authentication
Login to W&B for experiment tracking and metrics logging.

In [ ]:
!wandb login YOUR_WANDB_TOKEN

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


## 2. Imports & Configuration

### Import Libraries and Set Training Parameters
Importing all required libraries and configuring:
- Model paths and names
- Training hyperparameters
- Quantization settings
- LoRA/DoRA/AdaLoRA configurations

In [ ]:
# ====================================================================
# CELL 2: Imports and Configuration
# ====================================================================
import os
import json
import torch
import wandb
import gc
import time
import numpy as np
from PIL import Image
from transformers import (
    LlavaOnevisionForConditionalGeneration,
    LlavaOnevisionProcessor,
    BitsAndBytesConfig,
    AutoProcessor
)
from peft import LoraConfig, get_peft_model, AdaLoraConfig
from trl import SFTConfig, SFTTrainer
from huggingface_hub import HfApi
from google.colab import userdata

# Configuration
DATASET_PATH = "/kaggle/input/wsi-data-pfe/wsi-data-r/"  # UPDATE THIS
HF_USERNAME = "azdin"  # UPDATE THIS
MODEL_ID = "llava-hf/llava-onevision-qwen2-7b-ov-hf"
MAX_SAMPLES = 200  # Adjust 

# Get HF token from Colab secrets
try:
    HF_TOKEN = 'ur-token-here'  # Replace with your actual token or method to retrieve it
    print("✅ HF token loaded from secrets")
except:
    print("❌ Please add 'hf_token' to Colab secrets")
    HF_TOKEN = None

# Set environment
os.environ["TOKENIZERS_PARALLELISM"] = "false"

print("🚀 Setup completed!")

✅ HF token loaded from secrets
🚀 Setup completed!


### Utility Functions
Helper functions for:
- Memory management and GPU monitoring
- Dataset loading and preprocessing
- Model training and evaluation
- Inference and visualization

In [ ]:
# ====================================================================
# CELL 3: Utility Functions
# ====================================================================

def clear_memory():
    """Clear GPU memory"""
    gc.collect()
    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    print(f"GPU allocated: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
    print(f"GPU reserved: {torch.cuda.memory_reserved() / 1024**3:.2f} GB")

def load_weather_dataset(dataset_path, max_samples=None):
    """Load weather satellite dataset with metadata"""

    print(f"📂 Loading weather dataset from: {dataset_path}")

    # Load manifest
    manifest_path = os.path.join(dataset_path, "training_manifest.json")
    with open(manifest_path, 'r') as f:
        manifest = json.load(f)

    # Use all samples or limit
    if max_samples is None:
        data = manifest['training_data']
        print(f"Using ALL {len(data)} samples")
    else:
        data = manifest['training_data'][:max_samples]
        print(f"Limited to {len(data)} samples (max_samples={max_samples})")

    images_dir = os.path.join(dataset_path, "images")
    captions_dir = os.path.join(dataset_path, "captions")
    metadata_dir = os.path.join(dataset_path, "metadata")

    print(f"Processing {len(data)} samples with metadata integration...")

    def extract_temporal_context(metadata):
        """Extract temporal information from metadata"""
        temporal_info = {}

        if 'temporal_info' in metadata:
            temp_data = metadata['temporal_info']
            temporal_info = {
                'date': temp_data.get('date_formatted', 'Unknown date'),
                'season': temp_data.get('season', 'Unknown season'),
                'month': temp_data.get('month_name', 'Unknown month')
            }

        if 'image_info' in metadata and 'capture_date' in metadata['image_info']:
            temporal_info['date'] = metadata['image_info']['capture_date']

        return temporal_info

    def extract_weather_context(metadata):
        """Extract weather information"""
        weather_info = []

        if 'weather_descriptions' in metadata:
            for region_key, region_data in metadata['weather_descriptions'].items():
                raw_data = None

                if 'raw_data' in region_data:
                    raw_data = region_data['raw_data']
                elif 'regional_weather' in metadata and region_key in metadata['regional_weather']:
                    raw_data = metadata['regional_weather'][region_key].get('weather_12utc', {})

                weather_entry = {
                    'region': region_key,
                    'location': region_data.get('location', region_key),
                    'description': region_data.get('description', 'No description'),
                    'raw_data': raw_data or {}
                }
                weather_info.append(weather_entry)

        return weather_info

    def extract_city_coordinates(metadata):
        """Extract city coordinates"""
        cities_info = []
        if 'regional_weather' in metadata:
            for city_key, city_data in metadata['regional_weather'].items():
                if 'coordinates' in city_data:
                    coords = city_data['coordinates']
                    location_desc = city_data.get('location', city_key)
                    cities_info.append({
                        'name': city_key.replace('_', ' ').title(),
                        'description': location_desc,
                        'lat': coords['lat'],
                        'lon': coords['lon']
                    })
        return cities_info

    def format_weather_data(weather_context):
        """Format weather data table"""
        if not weather_context:
            return "Weather data unavailable"

        weather_table = "**SURFACE METEOROLOGICAL CONDITIONS (at 12:00 Noon UTC):**\n"
        weather_table += "| Location | Temp(°C) | Humidity(%) | Wind(m/s) | Precip(mm) | Cloud(%) | Pressure(kPa) |\n"
        weather_table += "|----------|----------|-------------|-----------|------------|----------|---------------|\n"

        for region_data in weather_context:
            if isinstance(region_data, dict) and 'raw_data' in region_data:
                raw = region_data['raw_data']
                location = region_data.get('location', 'Unknown').split(',')[0]
                temp = f"{raw.get('T2M', 'N/A'):.1f}" if raw.get('T2M') != 'N/A' else 'N/A'
                humidity = f"{raw.get('RH2M', 'N/A'):.0f}" if raw.get('RH2M') != 'N/A' else 'N/A'
                wind = f"{raw.get('WS10M', 'N/A'):.1f}" if raw.get('WS10M') != 'N/A' else 'N/A'
                precip = f"{raw.get('PRECTOTCORR', 'N/A'):.1f}" if raw.get('PRECTOTCORR') != 'N/A' else 'N/A'
                cloud = f"{raw.get('CLOUD_AMT', 'N/A'):.0f}" if raw.get('CLOUD_AMT') != 'N/A' else 'N/A'
                pressure = f"{raw.get('PS', 'N/A'):.0f}" if raw.get('PS') != 'N/A' else 'N/A'

                weather_table += f"| {location} | {temp} | {humidity} | {wind} | {precip} | {cloud} | {pressure} |\n"

        return weather_table

    def build_enhanced_system_prompt(metadata):
        """Build enhanced prompt used in GPT-4V collection"""

        temporal_context = extract_temporal_context(metadata)
        weather_context = extract_weather_context(metadata)
        weather_table = format_weather_data(weather_context)
        cities_info = extract_city_coordinates(metadata)

        # Build city reference section
        city_references = "**MOROCCAN CITIES (for geographic reference):**\n"
        for city in cities_info:
            city_name = city['name'].replace('_', ' ').title()
            if not any(word in city_name.lower() for word in ['mountains', 'sahara', 'desert', 'atlas', 'rif']):
                city_references += f"- **{city_name}** ({city['lat']:.2f}°N, {city['lon']:.2f}°W): {city['description']}\n"

        date_info = temporal_context.get('date', 'Unknown date')
        season_info = temporal_context.get('season', 'Unknown season')
        month_info = temporal_context.get('month', 'Unknown month')

        system_prompt = f"""You are a professional meteorologist and satellite imagery analyst with expertise in North African climatology and Morocco's atmospheric patterns, trained in international meteorological standards (WMO/NOAA/EUMETSAT protocols).

**SATELLITE ACQUISITION DETAILS:**
- **Date & Time**: {date_info} at 12:00 Noon UTC (Midday acquisition)
- **Geographic Domain**: Morocco, North Africa
- **Seasonal Context**: {season_info} ({month_info})
- **Satellite**: MODIS Terra - Corrected Reflectance True Color
- **Spatial Resolution**: 250 meters

{weather_table}

{city_references}

**ANALYSIS FRAMEWORK:**
Following standard meteorological satellite interpretation protocols for comprehensive weather analysis including:

1. **TEMPORAL AND GEOGRAPHIC CONTEXT:** Reference exact acquisition date, noon timing, and specific Moroccan cities
2. **SURFACE METEOROLOGICAL CONDITIONS:** Integrate ground-level weather measurements with satellite observations
3. **ATMOSPHERIC PHENOMENA IDENTIFICATION:** Cloud field analysis, weather systems, and dust analysis if visible
4. **CITY-FOCUSED WEATHER ANALYSIS:** Link atmospheric conditions to specific cities with ground-level measurements
5. **METEOROLOGICAL INTERPRETATION:** Atmospheric stability assessment and moisture distribution patterns

**CRITICAL INSTRUCTIONS:**
- ALWAYS begin with date and 12:00 noon UTC acquisition time
- ALWAYS reference specific Moroccan CITIES by name (not regions)
- ALWAYS specify "ground-level" or "surface" when mentioning weather measurements
- Keep descriptions concise, professional, and focused on observable atmospheric phenomena

Provide comprehensive meteorological analysis focusing on cities and ground-level weather integration."""

        return system_prompt

    def format_weather_sample_llava(item):
        """Format sample for LLaVA-OneVision"""

        # Load image
        image_path = os.path.join(images_dir, item['image_file'])
        image = Image.open(image_path).convert('RGB')

        # Load caption
        caption_path = os.path.join(captions_dir, item['caption_file'])
        with open(caption_path, 'r', encoding='utf-8') as f:
            caption = f.read().strip()

        # Load metadata and create enhanced system prompt
        system_message = """You are an expert meteorologist analyzing weather satellite imagery. Provide detailed, professional analysis of atmospheric conditions visible in MODIS Terra satellite images of Morocco."""

        if item.get('has_metadata') and item.get('metadata_file'):
            metadata_path = os.path.join(metadata_dir, item['metadata_file'])
            try:
                with open(metadata_path, 'r') as f:
                    metadata = json.load(f)

                system_message = build_enhanced_system_prompt(metadata)

            except Exception as e:
                print(f"⚠️ Could not load metadata for {item['image_file']}: {e}")

        # Format for LLaVA-OneVision
        # Combine system message with user query
        full_user_message = f"{system_message}\n\nPlease analyze this MODIS Terra satellite image of Morocco and provide a comprehensive meteorological analysis using the provided weather data and geographic context."

        return {
            "messages": [
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": full_user_message},
                        {"type": "image"}
                    ],
                },
                {
                    "role": "assistant",
                    "content": [
                        {"type": "text", "text": caption}
                    ],
                }
            ],
            "images": [image]
        }

    # Format all samples
    formatted_data = []
    for i, item in enumerate(data):
        try:
            sample = format_weather_sample_llava(item)
            formatted_data.append(sample)
            if i % 50 == 0:
                print(f"  Processed {i}/{len(data)} samples...")
        except Exception as e:
            print(f"  ⚠️ Skipped sample {i}: {e}")
            continue

    print(f"✅ Successfully formatted {len(formatted_data)} samples with metadata")
    return formatted_data

def create_data_collator_llava(processor):
    """Create data collator for LLaVA-OneVision"""
    def collate_fn(examples):
        # Extract messages and images
        texts = []
        images = []

        for example in examples:
            # Apply chat template
            text = processor.apply_chat_template(
                example["messages"],
                tokenize=False,
                add_generation_prompt=False
            )
            texts.append(text)
            images.append(example["images"][0])

        # Process batch
        batch = processor(
            text=texts,
            images=images,
            return_tensors="pt",
            padding=True
        )

        # Create labels
        labels = batch["input_ids"].clone()
        labels[labels == processor.tokenizer.pad_token_id] = -100

        # Mask image tokens in labels
        image_token_id = processor.tokenizer.convert_tokens_to_ids("<image>")
        labels[labels == image_token_id] = -100

        batch["labels"] = labels
        return batch

    return collate_fn

def upload_to_huggingface(model_path, repo_name, technique_name):
    """Upload model to Hugging Face Hub"""

    if not HF_TOKEN:
        print("❌ HF token not available, skipping upload")
        return None

    hf_repo_name = f"{HF_USERNAME}/{repo_name}"

    try:
        print(f"🚀 Uploading {technique_name} to HF Hub: {hf_repo_name}")

        # Initialize API
        api = HfApi(token=HF_TOKEN)

        # Create repo
        api.create_repo(
            repo_id=hf_repo_name,
            exist_ok=True,
            token=HF_TOKEN,
            private=False,
            repo_type="model"
        )

        # Upload model files
        api.upload_folder(
            folder_path=model_path,
            repo_id=hf_repo_name,
            commit_message=f"Upload LLaVA-OneVision weather satellite {technique_name} adapter"
        )

        # Create README
        readme_content = f"""---
license: apache-2.0
base_model: llava-hf/llava-onevision-qwen2-7b-ov-hf
tags:
- llava
- llava-onevision
- weather
- satellite
- morocco
- meteorology
- {technique_name.lower()}
- fine-tuned
---

# LLaVA-OneVision Weather Analysis - {technique_name}

Fine-tuned using **{technique_name}** technique for weather satellite imagery analysis.

## Model Details

- **Base Model:** llava-hf/llava-onevision-qwen2-7b-ov-hf
- **Technique:** {technique_name}
- **Domain:** Weather satellite imagery analysis
- **Dataset:** Weather satellite images with meteorological metadata

## Usage

```python
from transformers import LlavaOnevisionForConditionalGeneration, AutoProcessor
import torch

# Load base model
model = LlavaOnevisionForConditionalGeneration.from_pretrained(
    "llava-hf/llava-onevision-qwen2-7b-ov-hf",
    torch_dtype=torch.bfloat16,
    device_map="auto"
)
processor = AutoProcessor.from_pretrained("llava-hf/llava-onevision-qwen2-7b-ov-hf")

# Load fine-tuned adapter
model.load_adapter("{hf_repo_name}")

# Use for weather analysis...
```

## Training Details

- **Technique:** {technique_name}
- **Quantization:** 4-bit NF4
- **Training Data:** Weather satellite imagery with metadata
- **Target Modules:** Attention and projection layers
"""

        # Upload README
        api.upload_file(
            path_or_fileobj=readme_content.encode(),
            path_in_repo="README.md",
            repo_id=hf_repo_name,
            commit_message="Add README"
        )

        print(f"✅ Successfully uploaded to: https://huggingface.co/{hf_repo_name}")
        return hf_repo_name

    except Exception as e:
        print(f"❌ Upload failed: {e}")
        return None

print("🛠️ Utility functions loaded!")



🛠️ Utility functions loaded!


## 3. Dataset Preparation

### Load Weather Satellite Dataset
Loading and splitting the dataset into training and evaluation sets.

The dataset contains:
- Weather satellite images from various sources
- Descriptive captions for each image
- Meteorological annotations

In [ ]:
# ====================================================================
# CELL 4: Load Dataset
# ====================================================================

# Load dataset once for both techniques
print("📂 Loading weather dataset...")
dataset = load_weather_dataset(DATASET_PATH, max_samples=None)

# Split dataset
train_size = int(0.8 * len(dataset))
train_dataset = dataset[:train_size]
eval_dataset = dataset[train_size:]

print(f"📊 Dataset split: {len(train_dataset)} train, {len(eval_dataset)} eval")

📂 Loading weather dataset...
📂 Loading weather dataset from: /kaggle/input/wsi-data-pfe/wsi-data-r/
Using ALL 973 samples
Processing 973 samples with metadata integration...
  Processed 0/973 samples...
  Processed 50/973 samples...
  Processed 100/973 samples...
  Processed 150/973 samples...
  Processed 200/973 samples...
  Processed 250/973 samples...
  Processed 300/973 samples...
  Processed 350/973 samples...
  Processed 400/973 samples...
  Processed 450/973 samples...
  Processed 500/973 samples...
  Processed 550/973 samples...
  Processed 600/973 samples...
  Processed 650/973 samples...
  Processed 700/973 samples...
  Processed 750/973 samples...
  Processed 800/973 samples...
  Processed 850/973 samples...
  Processed 900/973 samples...
  Processed 950/973 samples...
✅ Successfully formatted 973 samples with metadata
📊 Dataset split: 778 train, 195 eval


---

## 4. Fine-Tuning Techniques

### Technique 1: DoRA (Weight-Decomposed Low-Rank Adaptation)

#### What is DoRA?
DoRA enhances LoRA by decomposing pre-trained weights into two components:
- **Magnitude**: The scale/strength of weights
- **Direction**: The orientation of weights

This decomposition allows the model to learn more effectively by updating both components separately.

#### Key Parameters:
- **Rank**: 16 (dimensionality of low-rank matrices)
- **Alpha**: 32 (scaling factor)
- **Target Modules**: q_proj, v_proj, k_proj, o_proj
- **Use DoRA**: True

#### Advantages:
✅ Better learning capacity than standard LoRA  
✅ More stable training dynamics  
✅ Superior performance on vision-language tasks  
✅ Minimal memory overhead compared to full fine-tuning

In [ ]:
# ====================================================================
# CELL 5: DoRA Technique Training
# ====================================================================

print("\n" + "="*60)
print("🔥 TECHNIQUE 1: DoRA (Weight-Decomposed LoRA)")
print("="*60)

# Setup W&B for DoRA
dora_config = {
    "technique": "DoRA",
    "model": "LLaVA-OneVision",
    "quantization": "4-bit NF4",
    "lora_rank": 16,
    "lora_alpha": 32,
    "target_modules": ["q_proj", "v_proj", "k_proj", "o_proj"],
    "samples": MAX_SAMPLES,
    "epochs": 3
}

wandb.init(
    project="llava-onevision-dora-weather",
    name="weather-satellite-dora",
    config=dora_config,
    tags=["dora", "llava-onevision", "weather-satellite", "comparison"]
)

clear_memory()

# Setup quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Load model
print("📥 Loading LLaVA-OneVision for DoRA...")
model = LlavaOnevisionForConditionalGeneration.from_pretrained(
    MODEL_ID,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config,
    low_cpu_mem_usage=True
)
processor = AutoProcessor.from_pretrained(MODEL_ID)

# DoRA config
try:
    peft_config = LoraConfig(
        lora_alpha=32,
        lora_dropout=0.05,
        r=16,
        bias="none",
        target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],
        task_type="CAUSAL_LM",
        use_dora=True,  # Enable DoRA
    )
    print("✅ DoRA configuration created")
except Exception as e:
    print(f"⚠️ DoRA not supported, using enhanced LoRA: {e}")
    peft_config = LoraConfig(
        lora_alpha=32,
        lora_dropout=0.05,
        r=16,
        bias="none",
        target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],
        task_type="CAUSAL_LM",
    )

# Apply PEFT
peft_model = get_peft_model(model, peft_config)
peft_model.print_trainable_parameters()

# Training arguments for DoRA
dora_output_dir = "./llava_dora_weather_model"
training_args = SFTConfig(
    output_dir=dora_output_dir,
    num_train_epochs=3,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=8,
    gradient_checkpointing=True,

    optim="adamw_torch",
    learning_rate=1e-4,  # Lower LR for DoRA
    lr_scheduler_type="constant",

    logging_steps=10,
    eval_steps=max(25, len(train_dataset) // 30),
    eval_strategy="steps",
    save_strategy="steps",
    save_steps=max(50, len(train_dataset) // 20),
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    load_best_model_at_end=True,

    bf16=True,
    tf32=False,
    max_grad_norm=0.3,
    warmup_ratio=0.03,

    push_to_hub=False,
    report_to="wandb",

    gradient_checkpointing_kwargs={"use_reentrant": False},
    dataset_text_field="",
    dataset_kwargs={"skip_prepare_dataset": True},
)

training_args.remove_unused_columns = False

# Create trainer
trainer = SFTTrainer(
    model=peft_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=create_data_collator_llava(processor),
    peft_config=peft_config,
    processing_class=processor.tokenizer,
)

# Train DoRA
print("🚀 Starting DoRA training...")
dora_start_time = time.time()

trainer.train()

dora_training_time = time.time() - dora_start_time

# Save model locally
trainer.save_model(dora_output_dir)

# Final evaluation
dora_eval_result = trainer.evaluate()

print(f"✅ DoRA training completed in {dora_training_time/60:.1f} minutes")
print(f"📊 Final DoRA eval loss: {dora_eval_result['eval_loss']:.4f}")

# Upload to HuggingFace
dora_hf_repo = upload_to_huggingface(dora_output_dir, "llava-onevision-weather-dora", "DoRA")

# Log final results
wandb.log({
    "final_training_time": dora_training_time,
    "final_eval_loss": dora_eval_result['eval_loss'],
    "hf_repo": dora_hf_repo,
    "training_completed": True
})

wandb.finish()

print(f"💾 DoRA model saved locally: {dora_output_dir}")
if dora_hf_repo:
    print(f"🌐 DoRA model on HF: https://huggingface.co/{dora_hf_repo}")




🔥 TECHNIQUE 1: DoRA (Weight-Decomposed LoRA)


wandb: Currently logged in as: azdinsahir11 (azdinsahir11-university-mohamed-v) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


GPU allocated: 0.00 GB
GPU reserved: 0.00 GB
📥 Loading LLaVA-OneVision for DoRA...


config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.91G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.23G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/126 [00:00<?, ?B/s]

### Technique 2: AdaLoRA (Adaptive Low-Rank Adaptation)

#### What is AdaLoRA?
AdaLoRA dynamically allocates the parameter budget across different weight matrices based on their importance during training.

**How it works:**
1. Starts with an initial rank (r_init)
2. Monitors importance of each adapter during training
3. Prunes less important adapters
4. Focuses parameters on critical layers

#### Key Parameters:
- **Initial Rank (init_r)**: 12
- **Target Rank (target_r)**: 8
- **Beta1/Beta2**: 0.85 (importance smoothing)
- **Tinit/Tfinal**: 200/1000 (adaptation schedule)

#### Advantages:
✅ Automatic parameter allocation  
✅ More efficient than fixed-rank LoRA  
✅ Adapts to task complexity  
✅ Reduces unnecessary parameters

In [ ]:
# ====================================================================
# CELL 6: AdaLoRA Technique Training
# ====================================================================

print("\n" + "="*60)
print("🔥 TECHNIQUE 2: AdaLoRA (Adaptive LoRA)")
print("="*60)

# Setup W&B for AdaLoRA
adalora_config = {
    "technique": "AdaLoRA",
    "model": "LLaVA-OneVision",
    "quantization": "4-bit NF4",
    "init_r": 12,
    "target_r": 8,
    "beta1": 0.85,
    "beta2": 0.85,
    "tinit": 200,
    "tfinal": 1000,
    "samples": MAX_SAMPLES,
    "epochs": 4
}

wandb.init(
    project="llava-onevision-adalora-weather",
    name="weather-satellite-adalora",
    config=adalora_config,
    tags=["adalora", "llava-onevision", "weather-satellite", "comparison"]
)

clear_memory()

# Setup quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Load model for AdaLoRA
print("📥 Loading LLaVA-OneVision for AdaLoRA...")
model = LlavaOnevisionForConditionalGeneration.from_pretrained(
    MODEL_ID,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config,
    low_cpu_mem_usage=True
)
processor = AutoProcessor.from_pretrained(MODEL_ID)

# AdaLoRA config
try:
    peft_config = AdaLoraConfig(
        init_r=12,
        target_r=8,
        beta1=0.85,
        beta2=0.85,
        tinit=200,
        tfinal=1000,
        deltaT=10,
        lora_alpha=16,
        lora_dropout=0.05,
        target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],
        task_type="CAUSAL_LM",
    )
    print("✅ AdaLoRA configuration created")
except Exception as e:
    print(f"⚠️ AdaLoRA not available, using adaptive LoRA: {e}")
    peft_config = LoraConfig(
        lora_alpha=24,
        lora_dropout=0.1,
        r=12,
        bias="none",
        target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],
        task_type="CAUSAL_LM",
    )

# Apply PEFT
peft_model = get_peft_model(model, peft_config)
peft_model.print_trainable_parameters()

# Training arguments for AdaLoRA
adalora_output_dir = "./llava_adalora_weather_model"
training_args = SFTConfig(
    output_dir=adalora_output_dir,
    num_train_epochs=4,  # More epochs for adaptation
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=8,
    gradient_checkpointing=True,

    optim="adamw_torch",
    learning_rate=3e-4,  # Higher LR for AdaLoRA
    lr_scheduler_type="cosine",  # Cosine schedule for adaptive learning

    logging_steps=10,
    eval_steps=max(20, len(train_dataset) // 40),
    eval_strategy="steps",
    save_strategy="steps",
    save_steps=max(40, len(train_dataset) // 25),
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    load_best_model_at_end=True,

    bf16=True,
    tf32=False,
    max_grad_norm=0.3,
    warmup_ratio=0.1,  # More warmup for adaptation

    push_to_hub=False,
    report_to="wandb",

    gradient_checkpointing_kwargs={"use_reentrant": False},
    dataset_text_field="",
    dataset_kwargs={"skip_prepare_dataset": True},
)

training_args.remove_unused_columns = False

# Create trainer
trainer = SFTTrainer(
    model=peft_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=create_data_collator_llava(processor),
    peft_config=peft_config,
    processing_class=processor.tokenizer,
)

# Train AdaLoRA
print("🚀 Starting AdaLoRA training...")
adalora_start_time = time.time()

trainer.train()

adalora_training_time = time.time() - adalora_start_time

# Save model locally
trainer.save_model(adalora_output_dir)

# Final evaluation
adalora_eval_result = trainer.evaluate()

print(f"✅ AdaLoRA training completed in {adalora_training_time/60:.1f} minutes")
print(f"📊 Final AdaLoRA eval loss: {adalora_eval_result['eval_loss']:.4f}")

# Upload to HuggingFace
adalora_hf_repo = upload_to_huggingface(adalora_output_dir, "llava-onevision-weather-adalora", "AdaLoRA")

# Log final results
wandb.log({
    "final_training_time": adalora_training_time,
    "final_eval_loss": adalora_eval_result['eval_loss'],
    "hf_repo": adalora_hf_repo,
    "training_completed": True
})

wandb.finish()

print(f"💾 AdaLoRA model saved locally: {adalora_output_dir}")
if adalora_hf_repo:
    print(f"🌐 AdaLoRA model on HF: https://huggingface.co/{adalora_hf_repo}")



### Technique 3: QLoRA (Quantized Low-Rank Adaptation)

#### What is QLoRA?
QLoRA combines quantization with LoRA to enable fine-tuning of large models on limited hardware.

**Key Innovation:**
- Base model weights: Quantized to 4-bit (NF4 format)
- LoRA adapters: Trained in full precision (16-bit/32-bit)
- Result: Massive memory savings with minimal quality loss

#### Key Parameters:
- **Rank**: 32 (higher rank compensates for quantization)
- **Alpha**: 64
- **Quantization**: 4-bit NormalFloat (NF4)
- **Compute dtype**: bfloat16

#### Advantages:
✅ Up to 75% memory reduction  
✅ Enables fine-tuning on consumer GPUs  
✅ Maintains model quality  
✅ Faster training with lower memory

In [ ]:
# ====================================================================
# CELL 9: QLoRA Technique Training
# ====================================================================

print("\n" + "="*60)
print("🔥 TECHNIQUE 3: QLoRA (Quantized LoRA)")
print("="*60)

# Setup W&B for QLoRA
qlora_config = {
    "technique": "QLoRA",
    "model": "LLaVA-OneVision",
    "quantization": "4-bit NF4",
    "lora_rank": 32,  # Higher rank for QLoRA
    "lora_alpha": 64,
    "target_modules": ["q_proj", "v_proj", "k_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],  # More modules
    "samples": MAX_SAMPLES,
    "epochs": 3,
    "double_quant": True,
    "compute_dtype": "bfloat16"
}

wandb.init(
    project="llava-onevision-qlora-weather",
    name="weather-satellite-qlora",
    config=qlora_config,
    tags=["qlora", "llava-onevision", "weather-satellite", "comparison"]
)

clear_memory()

# Setup enhanced quantization for QLoRA
qlora_bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,  # Key for QLoRA
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_storage=torch.bfloat16,  # Store quantized weights in bfloat16
)

# Load model for QLoRA
print("📥 Loading LLaVA-OneVision for QLoRA...")
model = LlavaOnevisionForConditionalGeneration.from_pretrained(
    MODEL_ID,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    quantization_config=qlora_bnb_config,
    low_cpu_mem_usage=True
)
processor = AutoProcessor.from_pretrained(MODEL_ID)

# Prepare model for k-bit training
from peft import prepare_model_for_kbit_training
model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=True)

# QLoRA config with enhanced settings
qlora_peft_config = LoraConfig(
    lora_alpha=64,
    lora_dropout=0.1,
    r=32,  # Higher rank for better capacity
    bias="none",
    target_modules=[
        "q_proj", "v_proj", "k_proj", "o_proj",  # Attention
        "gate_proj", "up_proj", "down_proj",     # MLP layers
        "embed_tokens",                           # Embeddings
        "lm_head"                                 # Output layer
    ],
    task_type="CAUSAL_LM",
    modules_to_save=["embed_tokens", "lm_head"],  # Save these modules in full precision
)

# Apply PEFT
peft_model = get_peft_model(model, qlora_peft_config)
peft_model.print_trainable_parameters()

# Training arguments optimized for QLoRA
qlora_output_dir = "./llava_qlora_weather_model"
training_args = SFTConfig(
    output_dir=qlora_output_dir,
    num_train_epochs=3,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=16,  # Higher accumulation for QLoRA
    gradient_checkpointing=True,

    optim="paged_adamw_32bit",  # Paged optimizer for QLoRA
    learning_rate=2e-4,
    lr_scheduler_type="cosine",

    logging_steps=10,
    eval_steps=max(30, len(train_dataset) // 25),
    eval_strategy="steps",
    save_strategy="steps",
    save_steps=max(60, len(train_dataset) // 15),
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    load_best_model_at_end=True,

    bf16=True,
    tf32=False,
    max_grad_norm=0.3,
    warmup_ratio=0.05,
    weight_decay=0.001,

    push_to_hub=False,
    report_to="wandb",

    # QLoRA specific settings
    ddp_find_unused_parameters=False,
    group_by_length=True,

    gradient_checkpointing_kwargs={"use_reentrant": False},
    dataset_text_field="",
    dataset_kwargs={"skip_prepare_dataset": True},
)

training_args.remove_unused_columns = False

# Create trainer with QLoRA optimizations
trainer = SFTTrainer(
    model=peft_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=create_data_collator_llava(processor),
    peft_config=qlora_peft_config,
    processing_class=processor.tokenizer,
)

# Train QLoRA
print("🚀 Starting QLoRA training...")
qlora_start_time = time.time()

trainer.train()

qlora_training_time = time.time() - qlora_start_time

# Save model locally
trainer.save_model(qlora_output_dir)

# Final evaluation
qlora_eval_result = trainer.evaluate()

print(f"✅ QLoRA training completed in {qlora_training_time/60:.1f} minutes")
print(f"📊 Final QLoRA eval loss: {qlora_eval_result['eval_loss']:.4f}")

# Upload to HuggingFace
qlora_hf_repo = upload_to_huggingface(qlora_output_dir, "llava-onevision-weather-qlora", "QLoRA")

# Log final results
wandb.log({
    "final_training_time": qlora_training_time,
    "final_eval_loss": qlora_eval_result['eval_loss'],
    "hf_repo": qlora_hf_repo,
    "training_completed": True
})

wandb.finish()

print(f"💾 QLoRA model saved locally: {qlora_output_dir}")
if qlora_hf_repo:
    print(f"🌐 QLoRA model on HF: https://huggingface.co/{qlora_hf_repo}")



## 5. Results & Comparison

### Training Comparison Summary
Compare the performance, speed, and resource usage of all three techniques.

In [ ]:
# ====================================================================
# CELL 7: Comparison Summary
# ====================================================================

print("\n" + "="*60)
print("📊 TRAINING COMPARISON SUMMARY")
print("="*60)
print(f"\n🔵 DoRA Results:")
print(f"   - Training Time: {dora_training_time/60:.1f} minutes")
print(f"   - Final Eval Loss: {dora_eval_result['eval_loss']:.4f}")
print(f"   - Model Path: {dora_output_dir}")

print(f"\n🟢 AdaLoRA Results:")
print(f"   - Training Time: {adalora_training_time/60:.1f} minutes")
print(f"   - Final Eval Loss: {adalora_eval_result['eval_loss']:.4f}")
print(f"   - Model Path: {adalora_output_dir}")

print("\n✅ Both models successfully trained and saved!")



## 6. Model Inference & Testing

### Test Fine-Tuned Models
Run inference with the fine-tuned models to evaluate their performance on sample images.

This cell demonstrates:
- Loading fine-tuned models
- Processing input images
- Generating descriptions
- Comparing outputs across techniques

In [ ]:
# ====================================================================
# CELL 8: Inference Example (Optional)
# ====================================================================

print("\n" + "="*60)
print("🔍 INFERENCE EXAMPLE")
print("="*60)

def run_inference_example(model_path, technique_name):
    """Run inference with fine-tuned model"""

    print(f"\n🎯 Testing {technique_name} model...")

    # Load the fine-tuned model
    model = LlavaOnevisionForConditionalGeneration.from_pretrained(
        MODEL_ID,
        device_map="auto",
        torch_dtype=torch.bfloat16,
        quantization_config=bnb_config
    )
    processor = AutoProcessor.from_pretrained(MODEL_ID)

    # Load adapter
    model.load_adapter(model_path)

    # Example inference
    example_prompt = """You are a professional meteorologist analyzing satellite imagery.

**SATELLITE ACQUISITION DETAILS:**
- Date & Time: January 15, 2025 at 12:00 Noon UTC
- Geographic Domain: Morocco, North Africa
- Satellite: MODIS Terra

Please analyze this satellite image and provide a meteorological analysis."""

    # Create dummy image for testing (replace with actual image)
    test_image = Image.new('RGB', (512, 512), color='blue')

    # Format input
    conversation = [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": example_prompt},
                {"type": "image"}
            ],
        }
    ]

    # Apply chat template
    prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)

    # Process inputs
    inputs = processor(text=prompt, images=[test_image], return_tensors="pt").to(model.device)

    # Generate
    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=512,
            do_sample=True,
            temperature=0.7,
            top_p=0.9
        )

    # Decode output
    response = processor.decode(output[0], skip_special_tokens=True)

    print(f"\n{technique_name} Response Preview:")
    print(response[:500] + "..." if len(response) > 500 else response)

    # Clean up
    del model
    clear_memory()

# Test both models
if os.path.exists(dora_output_dir):
    run_inference_example(dora_output_dir, "DoRA")

if os.path.exists(adalora_output_dir):
    run_inference_example(adalora_output_dir, "AdaLoRA")

print("\n🎉 All operations completed successfully!")

---

## Summary & Conclusions

### What We Accomplished
✅ Successfully fine-tuned LLaVA-OneVision for weather satellite analysis  
✅ Implemented and compared three PEFT techniques: DoRA, AdaLoRA, QLoRA  
✅ Evaluated models on weather satellite image descriptions  
✅ Compared performance, memory usage, and training efficiency  


**Documentation:**
- [LLaVA GitHub](https://github.com/haotian-liu/LLaVA)
- [HuggingFace PEFT](https://huggingface.co/docs/peft)
- [TRL Documentation](https://huggingface.co/docs/trl)

---

**Author**: Azeddin sahir 
